In [14]:
#Import de Tensor et vérification de la version
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
import tensorflow as tf
from pymongo import MongoClient


from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from tensorflow import keras
from tensorflow.keras import layers



c:\Users\yanni\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflowjs\read_weights.py:28: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  np.uint8, np.uint16, np.object, np.bool]


AttributeError: module 'numpy' has no attribute 'object'.
`np.object` was a deprecated alias for the builtin `object`. To avoid this error in existing code, use `object` by itself. Doing this will not modify any behavior and is safe. 
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [9]:

# Connexion à la base de données MongoDB
client = MongoClient("mongodb://localhost:27017")
db = client["DigitRecognizer"]
train_collection_name = "train"
test_collection_name = "test"

# Récupération des données d'entraînement et de test
train_cursor = db[train_collection_name].find()
test_cursor = db[test_collection_name].find()

# Conversion des données en DataFrames pandas
train_data = pd.DataFrame(list(train_cursor), columns=["label"] + [f"pixel{i}" for i in range(784)])

test_data = pd.DataFrame(list(test_cursor), columns=[f"pixel{i}" for i in range(784)])

#Ferme la connexion à Mongo
client.close()



In [20]:
# Diviser les données en features (X) et labels (y)
X = train_data.drop("label", axis=1)
y = train_data["label"]

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convertir les séries pandas en tableaux NumPy
X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values

# Normaliser les données
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialiser le modèle
model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(784,)),
    layers.Dense(10, activation='softmax')
])

# Compiler le modèle avec l'optimiseur Adam
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Entraîner le modèle
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Évaluer le modèle sur l'ensemble de test
accuracy = model.evaluate(X_test, y_test)[1]
print(f"Précision du modèle : {accuracy}")


Epoch 1/10
1050/1050 [==============================] - 4s 3ms/step - loss: 0.2854 - accuracy: 0.9207 - val_loss: 0.2058 - val_accuracy: 0.9479
Epoch 2/10
1050/1050 [==============================] - 3s 3ms/step - loss: 0.1135 - accuracy: 0.9662 - val_loss: 0.1969 - val_accuracy: 0.9564
Epoch 3/10
1050/1050 [==============================] - 3s 3ms/step - loss: 0.0686 - accuracy: 0.9789 - val_loss: 0.2046 - val_accuracy: 0.9604
Epoch 4/10
1050/1050 [==============================] - 3s 3ms/step - loss: 0.0507 - accuracy: 0.9844 - val_loss: 0.1975 - val_accuracy: 0.9618
Epoch 5/10
1050/1050 [==============================] - 3s 3ms/step - loss: 0.0370 - accuracy: 0.9884 - val_loss: 0.2436 - val_accuracy: 0.9611
Epoch 6/10
1050/1050 [==============================] - 4s 4ms/step - loss: 0.0257 - accuracy: 0.9924 - val_loss: 0.1826 - val_accuracy: 0.9633
Epoch 7/10
1050/1050 [==============================] - 4s 4ms/step - loss: 0.0214 - accuracy: 0.9935 - val_loss: 0.2076 - val_accuracy:

In [19]:
model_json = model.to_json()
with open("model.json","w") as json_file:
    json_file.write(model_json)
model.save_weights("model.h5")
